# Example of Advanced RAG pipeline

In this example we're going to build an Advanced RAG pipeline that can answer questions about the weather in a specific city. The example is going to use a couple of different techniques to make the pipeline more robust and efficient.
The techniques we're going to use are quite simple to use and can be applied to many different use cases. 

- Query Routing - Added flexibility to the pipeline by allowing it to dynamically select the right data source to answer the query.
- Hypothetical Document Embeddings (HyDE) - Generate hypothetical documents to allow for a broader search of the answers
- Reranking - Use a reranker to improve the quality the retrieved documents
- Custom Weather Query Engine - Use a custom query engine that fetches the weather data from an external API to use as context 

The following is a overview of the pipeline we're going to build:

![AdvancedRAGPipeline](./img/AdvancedRAGPipeline.png)

## Setup libraries and environment

In [ ]:
%pip install llama-index-postprocessor-rankgpt-rerank llama-index-postprocessor-cohere-rerank

In [ ]:
import os

from dotenv import load_dotenv
from IPython.display import Markdown, display
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.prompts.default_prompt_selectors import \
    DEFAULT_TREE_SUMMARIZE_PROMPT_SEL
from llama_index.core.query_engine import (RouterQueryEngine,
                                           TransformQueryEngine)
from llama_index.core.response_synthesizers import TreeSummarize
from llama_index.core.selectors import LLMMultiSelector
from llama_index.core.tools import QueryEngineTool
from llama_index.llms.openai import OpenAI
from llama_index.postprocessor.cohere_rerank import CohereRerank

from util.helpers import create_and_save_wiki_md_files, get_wiki_pages
from util.query_engines import VerboseHyDEQueryTransform, WeatherQueryEngine

In [ ]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio
nest_asyncio.apply()

Add the following to a `.env` file in the root of the project if not already there.

```
OPENAI_API_KEY=<YOUR_KEY_HERE>
```

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

#### Define custom `WeatherQueryEngine` 

In [ ]:
weather_query_engine = WeatherQueryEngine(verbose=True)

#### Define cities query engine

We define a query engine using HyDE to transform the initial query and Cohere Rerank to rerank the retrieved documents

In [ ]:
cities_pages = get_wiki_pages(
    [
        "Aarhus",
        "London",
        "Paris",
        "Berlin",
        "Tokyo",
        "Beijing",
        "Moscow",
        "Sydney",
    ]
)
create_and_save_wiki_md_files(cities_pages, path="./data/docs/cities/")


In [ ]:

cities_documents = SimpleDirectoryReader("./data/docs/cities").load_data()

cities_index = VectorStoreIndex.from_documents(cities_documents, show_progress=True)

reranker = CohereRerank(api_key=COHERE_API_KEY, top_n=5, model="rerank-english-v3.0")
hyde = VerboseHyDEQueryTransform(include_original=True, verbose=True)
cities_query_engine = TransformQueryEngine(
    query_engine=cities_index.as_query_engine(
        similarity_top_k=10, node_postprocessors=[reranker], verbose=True
    ),
    query_transform=hyde,
)

#### Define `RouterQueryEngine`

In [ ]:
weather_tool = QueryEngineTool.from_defaults(
    query_engine=weather_query_engine,
    description="Useful for getting todays weather forecast for a given city",
)
cities_tool = QueryEngineTool.from_defaults(
    query_engine=cities_query_engine,
    name="Cities Wiki Pages",
    description="Useful for getting information about cities",
)

llm = OpenAI(api_key=OPENAI_API_KEY, model="gpt-4-turbo")
query_engine = RouterQueryEngine(
    selector=LLMMultiSelector.from_defaults(llm=llm),
    llm=llm,
    summarizer=TreeSummarize(
        llm=llm, summary_template=DEFAULT_TREE_SUMMARIZE_PROMPT_SEL, verbose=True
    ),
    query_engine_tools=[
        weather_tool,
        cities_tool,
    ],
    verbose=True,
)

In [ ]:
response = await query_engine.aquery("At what time should I go running today in Paris?")
display(Markdown(f'{response}'))

In [ ]:
response = await query_engine.aquery("What are the best sights to see in Aarhus?")
display(Markdown(f'{response}'))

In [ ]:
response = await query_engine.aquery(
    "What are the best sights to see in Aarhus and at what time is it best to go there with regards to weather?"
)
display(Markdown(f"{response}"))